# Using matplotlib to visualize data

In [29]:
import db
import parser
import pandas as pd
from datetime import datetime, timedelta

import seaborn as sns
sns.set_theme()

In [80]:
dev = db.dev()
begin = datetime.today() + timedelta(days=-30)
data = list(
    dev.AppUpdateLog.find({"createdAt": {"$gt": begin}})
)
parser.pretty(data)

<IPython.core.display.JSON object>

In [130]:
# Explode array
df = pd.json_normalize(data, "log", record_prefix="log.", meta=['userId', 'clientType', 'status'])

In [82]:
df = pd.DataFrame({
    'timestamp': df['log.timestamp'], 
    'user': df['userId'],
    'os': df['clientType'],
    'version': df['log.clientVersion'],
    'status': df['log.status'],
    'finalStatus': df['status']
})
df['date'] = df['timestamp'].dt.date

In [96]:
df.groupby(['date', 'os', 'version', 'status']).count()

timestamp  user  finalStatus
date       os  version status                                 
2022-09-22 ios 2.1.0   rejected           4     4            4
               2.2.18  rejected           2     2            2
               2.5.0   succeeded          1     1            1
           web 2.3.0   rejected           1     1            1
               2.3.3   rejected          15    15           15
                       succeeded          1     1            1
               2.3.4   succeeded          2     2            2
2022-09-27 ios 2.2.18  rejected           1     1            1
2022-09-30 ios 2.1.0   rejected           1     1            1

In [129]:
df2 = pd.DataFrame(data)
df2 = pd.DataFrame({
    'date': df2['updatedAt'].dt.date,
    'os': df2['clientType'],
    'status': df2['status']
})
df2.groupby(['date', 'status']).count()

os
date       status       
2022-09-22 rejected    1
           succeeded   4
2022-09-27 rejected    1
2022-09-30 rejected    1

In [ ]:
# Add column
# df["attempt"] = df["logIndex"] + 1

# Remove column
# df=df.drop(["_id", "logIndex"], axis=1)

# Reposition column
# df.insert(2, 'attempt', df.pop('attempt'))

# Rename and reorder
# pd.DataFrame({'timestamp': df['log.timestamp'], 'user': df['userId']})